# Procesamiento de ficheros con grandes cantidades de datos en Python 

# Generamos todos los ficheros en caso de que no los tengas descargados

In [ ]:
from src.utils import generate_sample_data
from pathlib import Path
import shutil

In [ ]:
path = Path('.')
(path / 'data').mkdir(mode=0o775, exist_ok=True)
path_data = path / 'data'

filename_ = 'sample50M'

La celda inferior usará una función de utils.py para generar los ficheros con 50 millones de filas de datos aleatorios. 

Aunque hayas descargado los archivos con anterioridad, ejecuta igualmente la celda. Si están bien descomprimidos, te dará un mensaje indicando que el archivo ya existe.

``` File data/sample50M.csv already exists. Not created!! ```

In [ ]:
# Ficheros CSV de 50M de filas. 
generate_sample_data(fmt='csv', filename=f'{str(path_data)}/{filename_}')
generate_sample_data(fmt='csv', filename=f'{str(path_data)}/{filename_}', compression= 'gzip')

# Ficheros Parquet de 50M de filas. 
generate_sample_data(fmt='parquet', filename=f'{str(path_data)}/{filename_}')
generate_sample_data(fmt='parquet', filename=f'{str(path_data)}/{filename_}', compression = 'gzip')

# Ficheros Feather de 50M de filas.
generate_sample_data(fmt='feather', filename=f'{str(path_data)}/{filename_}')
# ¿Por qué no generamos el feather.zip?

Los ficheros generados contienen los mismos datos, que consisten en dos columnas:
* **product**: tipo de producto, que puede ser A, B, C, D, E o F.
* **price**: número entero que representa el precio al que se compró el producto.

# Almacenamiento en Memorias Secundarias

En el análisis de grandes volúmenes de datos, el formato en el que se almacenan en la memoria secundaria juega un papel crucial cuando necesitamos cargar esos datos en la memoria principal. En esta presentación, explicaremos varios formatos de almacenamiento que utilizaremos a lo largo del tutorial, analizando sus ventajas y desventajas principales. 

Finalmente, examinaremos cómo estos formatos y técnicas de compresión interactúan con el funcionamiento de la memoria principal del ordenador (RAM), para poder determinar las mejores estrategias a la hora de  manejar ficheros con grandes cantidades de datos en Python.

Recordar que en todo momento haremos referencia al concepto de **"formatos de memoria secundaria"** como los formatos de escritura en disco.

## 1. Principales formatos de ficheros para grandes cantidades de datos

### 1.1 Formatos Textuales

Los formatos textuales para datos son estructuras de almacenamiento y representación de información que utilizan texto plano, compuesto por caracteres legibles a través de un simple editor de texto. A diferencia de los formatos binarios, que almacenan información en secuencias de bits no directamente comprensibles para los humanos, los formatos textuales emplean caracteres ASCII o Unicode (UTF-8, UTF-16, etc.), lo que permite que los datos sean editados, visualizados y comprendidos sin necesidad de herramientas especializadas o conocimiento técnico avanzado. Esta accesibilidad es el principal motivo por el cual es el tipo de formato más extendido a la hora de almacenar datos.

Entre los formatos textuales más comunes están **CSV** para datos tabulares, **JSON** para datos jerárquicos en aplicaciones web, **XML** para configuraciones y servicios web, y **YAML** para archivos de configuración. Cada formato se elige según el tipo de datos y los requisitos específicos del sistema.

#### Ventajas de los formatos textuales:

- **Fácil legibilidad:**  
  Como hemos comentado, los formatos textuales utilizan texto plano para almacenar la información, lo que los hace comprensibles al ojo humano mediante el editor de texto que viene instalado por defecto en cualquier sistema. 

- **Estructura flexible, pero no descriptivos por sí solos:**  
  Muchos de los formatos (como JSON y XML) permiten representar estructuras anidadas de forma fácil y sin necesidad de predefinir un esquema, lo que les otorga gran flexibilidad para manejar datos complejos. Sin embargo, esta flexibilidad implica que no siempre contienen información clara sobre los tipos de datos o la validación de su contenido, requiriendo herramientas adicionales (JSON Schema, XML Schema Definition (XSD),... ) para garantizar una descripción precisa.

- **Portabilidad y compatibilidad:**  
  Los formatos textuales, al estar basados en texto plano, son ampliamente compatibles con diferentes sistemas operativos y plataformas. La mayoría de los equipos ya incluyen herramientas de edición de texto (como Notepad, nano o vim), lo que facilita su uso sin necesidad de instalar software adicional. Por lo tanto suelen ser los formatos más extendidos ya que son fácilmente manipulables con las herramientas preinstaladas.  


#### Desventajas de los formatos textuales:

- **Tamaño del archivo:**  
  Los formatos textuales suelen ser más grandes que los formatos binarios equivalentes al almacenar los datos en texto plano, lo que requiere más caracteres y, por lo tanto, más espacio de almacenamiento. Esto puede ser una limitación significativa cuando se trabaja con grandes volúmenes de datos, ya que el tamaño del archivo afecta tanto al almacenamiento como al rendimiento de la transmisión de datos.

- **Falta de eficiencia de procesamiento:**  
  A la hora de acceder a datos almacenados en formatos textuales, estos deben ser convertidos a los tipos de datos adecuados en memoria (por ejemplo, números, fechas), lo que agrega una sobrecarga de procesamiento. Además, la necesidad de análisis sintáctico (Parsing) para interpretar el texto puede aumentar el tiempo de lectura y escritura.

- **Consumo de memoria:**  
  Cuando se procesan archivos textuales grandes, se puede requerir una cantidad significativa de memoria, especialmente si el archivo completo se carga en la memoria para su manipulación. Esto puede llevar a problemas de rendimiento o incluso fallos en sistemas con recursos limitados de memoria. Además, los programas estándar para leer estos formatos suelen ser también los menos óptimos. 

- **Falta de estandarización (en algunos formatos):**  
  No todos los formatos textuales tienen una especificación estricta o estándar, lo que puede llevar a problemas de interoperabilidad. Por ejemplo, el formato **CSV** no tiene un estándar único, pueden variar en delimitadores, manejo de comillas y codificación de caracteres, lo que puede causar errores al compartir estos archivos con otros sistemas. Además, no tienen una forma nativa de representar valores faltantes (NaN, None, Null), lo que implica que la gestión de datos incompletos es ambigua y dependiente del contexto, por lo que es necesario usar documentación adicional 

- **Seguridad:**  
  Los archivos textuales pueden ser más susceptibles a problemas de seguridad, como inyecciones de scripts o manipulación maliciosa de datos, debido a su naturaleza legible y editable.

- **Limitaciones en la representación de datos complejos:**  
  Aunque formatos como **JSON** y **XML** permiten estructuras jerárquicas, los formatos textuales en general no son ideales para representar datos muy complejos que requieren un almacenamiento más eficiente. Además, manipular estructuras muy anidadas o complejas en estos formatos puede volverse complicado y propenso a errores.

Todo esto puede hacer que el uso de estos formatos sea ambiguo e incompleto a la hora de trabajar con grandes cantidades de datos donde es necesario mantener una rigurosidad en la interpretación de la estructura, tipos de datos y relaciones. La falta de un esquema predefinido o de documentación detallada puede causar problemas de coherencia y derivar en análisis erróneos.

### 1.2 Formatos Binarios

Los formatos binarios para datos son estructuras de almacenamiento que representan la información en secuencias de bits directamente comprensibles por las máquinas. A diferencia de los formatos textuales, que utilizan caracteres legibles por humanos, los formatos binarios almacenan datos de manera más compacta y eficiente, utilizando codificaciones específicas para tipos de datos como enteros, flotantes, imágenes, audio y video. Esta codificación directa permite una manipulación y un procesamiento más rápidos, especialmente cuando se manejan grandes volúmenes de datos o datos de alta complejidad. Debido a su eficiencia, los formatos binarios son más usados en procesos que requieren alto rendimiento.

Entre los formatos binarios más comunes dentro del análisis de datos se encuentran **Parquet** y **Feather** para datos estructurados orientados a columnas, y **Avro** para datos orientados a filas.

#### Ventajas de los formatos binarios:

- **Alta eficiencia de almacenamiento:**  
  Los formatos binarios almacenan datos de manera más compacta y eficiente que los formatos textuales. Al codificar directamente los tipos de datos en bits, reducen el tamaño de los archivos y optimizan el uso del almacenamiento. Esto es especialmente beneficioso para aplicaciones que manejan grandes volúmenes de datos o donde el espacio de almacenamiento es crítico.

- **Mayor velocidad de procesamiento:**  
  Debido a su estructura optimizada, los formatos binarios permiten un acceso y procesamiento de datos mucho más rápido. Los datos se leen y escriben directamente en formatos que los ordenadores pueden procesar sin necesidad de conversión adicional, lo que reduce la sobrecarga de la CPU y mejora el rendimiento general.

- **Menor consumo de memoria:**  
  Los formatos binarios están diseñados para ser eficientes en el uso de la memoria. Al representar datos en un formato comprimido y sin la sobrecarga adicional de caracteres de texto, se requiere menos memoria para cargar y manipular los datos. Esto es crucial en aplicaciones de alta demanda donde el rendimiento y la eficiencia de la memoria son esenciales.

- **Seguridad:**  
  Los datos en formatos binarios son menos susceptibles a manipulación accidental o maliciosa en comparación con los formatos textuales. Sin embargo, no son inmunes a manipulaciones o ataques maliciosos. La seguridad de estos datos depende fundamentalmente de la complejidad y robustez de las herramientas de lectura y escritura utilizadas, más que del hecho de ser un formato binario.

- **Adecuados para datos complejos:**  
  Los formatos binarios son ideales para almacenar y transmitir datos complejos, como imágenes, audio, video y grandes conjuntos de datos. Permiten una representación eficiente y exacta de estructuras de datos complejas, manteniendo la integridad y precisión de los datos.

#### Desventajas de los formatos binarios:

- **Falta de legibilidad humana:**  
  A diferencia de los formatos textuales, los formatos binarios no son legibles y comprensibles para los humanos sin el uso de software específico que no suele ir preinstalado en los sistemas comunes, lo que dificulta la inspección manual.

- **Dificultad en la manipulación y edición:**  
  Modificar datos en un formato binario requiere herramientas especializadas o conocimientos técnicos "avanzados", lo que puede complicar la edición y manipulación directa de los datos. Esto es una desventaja en entornos de trabajo donde los datos deben modificarse rápidamente y de manera sencilla.

- **Portabilidad limitada entre plataformas:**  
  Aunque los formatos binarios son eficientes, no siempre son tan portables como los formatos textuales. Diferentes sistemas operativos o aplicaciones pueden tener incompatibilidades con ciertos formatos binarios, lo que puede requerir conversiones o herramientas adicionales para la interoperabilidad.

- **Riesgo de corrupción de datos:**  
  En los formatos binarios, incluso una pequeña corrupción de datos puede causar errores significativos o hacer que todo el archivo sea ilegible, ya que la estructura de datos es más sensible a errores en comparación con los formatos textuales.

- **Mayor complejidad en la representación de datos sencillos:**  
  Para datos simples o aplicaciones que no requieren alta eficiencia o rendimiento, los formatos binarios pueden ser innecesariamente complejos. En tales casos, los formatos textuales pueden ser más adecuados por su simplicidad y facilidad de uso.

### 2. Principales formatos de almacenamiento de datos por columnas y filas.

Dentro del almacenamiento de datos puede organizarse principalmente en dos formatos: por columnas y por filas. Los formatos de almacenamiento por filas son ideales para operaciones donde es necesario leer o escribir registros completos, mientras que los formatos de almacenamiento por columnas son más eficientes para operaciones analíticas, donde solo se necesita acceder a ciertas columnas.

Con un ejemplo visual, podemos ver la diferencia entre ambos formatos:

![SNOWFALL](images/estructura.png) 

Los formatos de almacenamiento de datos por columnas permiten además que la compresión sea mucho más efectiva, puesto que los datos con tipos de datos similares están situados de forma consecutiva. Y si además, hemos ordenado esa columna, la compresión será todavía mayor.


#### 2.1 Comparativa entre formatos. 

**Formato** | **Compresión nativa** | **Ventajas** | **Desventajas** | **Tipos de Datos Soportados**
--- | --- | --- | --- | ---
**CSV** | No | Fácil de usar, ampliamente soportado | Ocupa mucho espacio en disco y memoria, especialmente con grandes volúmenes de datos | Texto sin formato, convierte todo a cadenas de texto por defecto; puedes especificar `int32`, `int64`, `float64`, etc., pero no es eficiente en la interpretación de tipos y no existe un estándar.
**Apache Parquet** | Sí | Almacenamiento eficiente, ideal para operaciones analíticas, ocupa menos espacio en memoria | No adecuado para insertar o modificar registros de forma frecuente | Soporta una amplia gama de tipos: `int32`, `int64`, `float`, `double`, `boolean`, `string`, `timestamp`, y tipos complejos como listas y mapas
**Feather** | Opcional (LZ4 o ZSTD) | Muy rápido para leer y escribir, excelente para análisis en memoria | Ocasionalmente ocupa más espacio que Parquet, no optimizado para sistemas distribuidos | Soporta tipos nativos de `pandas` y `numpy` como `int32`, `int64`, `float64`, `boolean`, `categorical`, `datetime`, etc.

### 2.2 ¿Por qué el CSV es el formato más extendido?

- Es el formato más utilizado debido a su simplicidad, compatibilidad y facilidad de uso.
- Al ser texto plano, puede ser abierto y modificado con casi cualquier herramienta, desde editores de texto hasta hojas de cálculo como Excel.
- Es una opción preferida para compartir datos entre sistemas y usuarios con diferentes niveles de experiencia técnica.
- A pesar de ser el más usado, está lejos de ser el más óptimo, como veremos en este tutorial.
- En definitiva es el más extendido por que se puede abrir con Excel. 

### 2.3 ¿Por qué Parquet ocupa menos en memoria secundaria?

- Parquet tiene **compresión nativa**, lo que significa que los datos se almacenan comprimidos dentro del archivo, reduciendo significativamente el tamaño en disco y en memoria.
- Al ser un formato **columnar**, comprime las columnas de forma independiente, haciendo más eficiente la compresión de datos del mismo tipo.
- Aplica técnicas de codificación como **Run-Length Encoding (RLE)** y **codificación por diccionario** para comprimir valores repetidos.
  
Más adelante en este tutorial, veremos por qué Parquet también es muy eficiente para realizar operaciones.

### 2.4 ¿Por qué es tan rápido en lectura y escritura Feather?

- Feather utiliza el formato de lectura de memoria principal **Apache Arrow**.
- Apache Arrow está optimizado para el **acceso aleatorio** y la manipulación rápida de grandes volúmenes de datos.
- Organiza los datos en un formato **columnar**, lo que permite acceder solo a las columnas necesarias sin cargar todo el conjunto de datos, mejorando el rendimiento.
- A diferencia de otros formatos, Feather **no prioriza la compresión**, lo que reduce el tiempo de procesamiento, haciéndolo ideal para operaciones que requieren lecturas y escrituras rápidas.
- Esto convierte a Feather en una opción ideal para **análisis interactivos** y tareas que requieren acceso frecuente a los datos.

También podéis consultar: 

https://arrow.apache.org/docs/python/feather.html

https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_feather.html

### 3 Gestión de Memoria y Movimiento de Datos en el Sistema (RAM y Cahache) 

![SNOWFALL](images/diagrama.png) 

| **Tipo de Memoria**      | **Capacidad**                | **Velocidad**         | **Latencia Aproximada** | **Función**                                                                                       |
|--------------------------|------------------------------|-----------------------|------------------------------------|---------------------------------------------------------------------------------------------------|
| **Caché L1**              | Muy pequeña (32KB - 128KB)    | Extremadamente rápida  | 0.33 - 1 ns (nanosegundos)         | Almacena los datos más usados por la CPU. Es la más rápida pero con muy poca capacidad.            |
| **Caché L2**              | Pequeña (256KB - 1MB)         | Muy rápida             | 3.3 - 10 ns                       | Almacena datos usados frecuentemente pero no tan recurrentes como en L1, los datos llegas descompirimido o parcialmente comprimidos.                           |
| **Caché L3**              | Mayor (2MB - 16MB)            | Rápida                | 10 - 33 ns                        | Compartida entre núcleos de CPU, se usa para almacenar datos menos frecuentes que en L1 y L2, los datos pueden estar comprimidos.      |
| **RAM (Memoria Principal)**| Grande (GBs)                 | Moderadamente rápida   | 33 - 100 ns                       | Almacena temporalmente programas y datos en uso por el sistema, pero es más lenta que las cachés.  |
| **Disco (HDD/SSD)**       | Muy grande (GBs a TBs)        | Muy lenta (comparada con RAM) | 3.3 - 33 µs (microsegundos)       | Almacena datos permanentes, pero es mucho más lento que la RAM y las cachés.                      |


Como podemos ver en el cuadro resumen, las cachés L1 y L2 son las memorias más rápidas en la arquitectura de un sistema, pero también son las más pequeñas. La caché L1 ofrece una latencia extremadamente baja, lo que significa que los cálculos se realizan de manera casi inmediata si los datos que necesita el procesador están almacenados allí. La caché L2, aunque un poco más lenta que L1, sigue siendo mucho más rápida que la RAM o el acceso a disco. Dado que las cachés L1 y L2 son pequeñas y particulares de cada núcleo, es crucial que mantengamos la mayor cantidad posible de información útil en L3 para maximizar la eficiencia.

#### 3.1 Relación con la elección de tipos de datos
Cuando eliges tipos de datos más pequeños y específicos, como int32 en lugar de int64, estás optimizando el uso de memoria y, de forma crucial, el uso de las cachés.
Supongamos que estás calculando la suma de edades de millones de registros. Si utilizas int32 para representar las edades, cada valor ocupa 4 bytes. Sin embargo, si utilizas int64, cada valor ocupa 8 bytes, el doble de espacio.

En la caché L1, que es la más rápida pero también la más pequeña, un tamaño típico es entre 32KB y 128KB.

Con int32: Si un valor de edad ocupa 4 bytes, podrías almacenar más valores en la caché L1 al mismo tiempo. Por ejemplo, en una caché L1 de 64KB, puedes almacenar hasta 16,384 valores int32.

Con int64: Si un valor ocupa 8 bytes, podrías almacenar la mitad de esos valores en la misma caché L1. En una caché L1 de 64KB, podrías almacenar solo 8,192 valores int64.

Optimizando el tipo de dato que necesitamos evitamos:

Que el procesador tenga que volver a completar ciclos de trabajo para conseguir los datos que necesita y que no están ni en la caché ni en la RAM.
Que tenga que acceder a la memoria RAM (más lenta que la caché) para obtener dichos datos.

# ¡Se acabó la turra, vayamos a ver código de una vez!
